In [14]:
import tkinter as tk
import cv2
import PIL
import PIL.Image, PIL.ImageTk
import numpy as np
from tkinter.ttk import *
from tkinter import *
from tkinter.filedialog import askdirectory
from tkinter import messagebox
import os

In [15]:
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
import pathlib

In [16]:
if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
         os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile


In [ ]:
class Runner(tk.Tk):


    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        container = tk.Frame(self)

        container.pack(side="top", fill="both", expand=True)
        self.frames = {}

        frame = StartPage(container, self)

        self.frames[StartPage] = frame
        frame.grid(row=0, column=0, sticky="nsew")
        self.show_frame(StartPage)

    def show_frame(self, cont):
        frame = self.frames[cont]
        frame.tkraise()
        
class StartPage(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.folder_path = None
        self.images_names_list = []
        self.image_index = -1
        self.img_width = 1000
        self.img_height = 800
        self.threshold = 0.7
        
        # tensorflow
        self.Object_detection_init()
        
        #filter variable
        self.filter_label_list = [1]*len(self.category_index)
        self.label_dis = {}
        self.index_to_label_id = {}
        self.filter_id = []
        m = 0
        for i in self.category_index:
            self.label_dis[self.category_index[i]["name"]] = Variable()
            self.index_to_label_id[m] = self.category_index[i]["id"]
            m += 1
        #frame 1 
        buttons_frame = tk.Frame(self)
        self.btn_open=Button(buttons_frame, text="Open\nFolder",width=10,command = lambda:self.open_file())
        self.btn_open.pack(anchor=tk.CENTER,padx =30,pady =30)
        self.btn_next=Button(buttons_frame, text="Next\nImage", width=10,command = lambda:self.next_image())
        self.btn_next.pack(anchor=tk.CENTER,padx =30,pady =30)
        self.btn_prev=Button(buttons_frame, text="Previous\nImage", width=10,command = lambda:self.prev_image())
        self.btn_prev.pack(anchor=tk.CENTER,padx =30,pady =30)
        self.btn_annot=Button(buttons_frame, text="Save\nAnnotation", width=10)
        self.btn_annot.pack(anchor=tk.CENTER,padx =30,pady =30)
        buttons_frame.pack(pady=30,side="left",anchor="nw")
        
        #frame 2
        cap_frame2 = tk.Frame(self)
        self.canvas1 = tk.Canvas(cap_frame2, width = self.img_width, height =self.img_height,bg="black",bd=2)
        self.canvas1.pack(padx=5,pady=5)
        cap_frame2.pack(side="left",anchor="nw",pady=30)
        
        #frame 3
        cap_frame3 = tk.Frame(self)
        label = tk.Label(cap_frame3, text="Select Model")
        label.pack(pady = 10,padx = 20)
        self.variable = tk.StringVar(cap_frame3)
        self.models = ["FRCNN","SSD"]
        self.variable.set("FRCNN")
        self.w = OptionMenu(cap_frame3, self.variable,*self.models)
        self.w.pack(pady = 10,padx = 20)
        self.detector=Button(cap_frame3, text="DETECT", width=10,command = lambda:self.detect())
        self.detector.pack(anchor=tk.CENTER,padx =30,pady =40)
        self.threshold_var = DoubleVar()
        s = Scale(cap_frame3, label="Set Threshold", orient=HORIZONTAL,from_=0, to=1, resolution=0.05,variable=self.threshold_var)
        s.set(0.7)
        s.pack()
        self.detection_thresh=Button(cap_frame3, text="Detection\nThreshold", width=10,command = lambda:self.changeThreshold())
        self.detection_thresh.pack(anchor=tk.CENTER,padx =30,pady =5)
        
        # Check box and Scrollbar 
        self.vsb = tk.Scrollbar(cap_frame3, orient="vertical")
        self.text = tk.Text(cap_frame3, width=25, height=45,yscrollcommand=self.vsb.set)
        self.vsb.config(command=self.text.yview)
        self.vsb.pack(side="right", fill="y")
        self.text.pack(side="left", fill="both", expand=True)
        for nm in self.label_dis:
            cb = tk.Checkbutton(cap_frame3, text=nm, variable=self.label_dis[nm],onvalue = 1, offvalue = 0,
                                command=lambda:self.label_filter())
            self.label_dis[nm].set(0)
            self.text.window_create("end", window=cb)
            self.text.insert("end", "\n") # to force one checkbox per line

        cap_frame3.pack(side="left",anchor="nw",pady=30)
        
    def Object_detection_init(self):
        #tensorflow 
        CWD_PATH = "/home/aayush/Downloads/tensorflow_object_detection_api/models/research/object_detection/"
        self.model_name = 'faster_rcnn_inception_v2_coco_2018_01_28'
        PATH_TO_CKPT = os.path.join(CWD_PATH, self.model_name, "frozen_inference_graph.pb")
        self.PATH_TO_LABELS = os.path.join(CWD_PATH,"data",'mscoco_label_map.pbtxt')
        self.category_index = label_map_util.create_category_index_from_labelmap(self.PATH_TO_LABELS, use_display_name=True)
        self.detection_model = self.load_model(self.model_name)
        
    def changeThreshold(self):
        self.threshold = self.threshold_var.get()
        self.detect()
    
    def label_filter(self):
        g = 0
        for lab in self.label_dis:
            self.filter_label_list[g] = self.label_dis[lab].get()
            g += 1
        filter_index = [j for j in range(len(self.filter_label_list)) if self.filter_label_list[j] == 1]
        if len(filter_index) == 0:
            messagebox.showinfo("Warning", "No label is selected. please select label.")
        self.filter_id = [self.index_to_label_id[k] for k in filter_index]
        
    def open_file(self): 
        self.folder_path =askdirectory()
        if self.folder_path:
            for name in os.listdir(self.folder_path):
                if name.split(".")[-1].lower() in ["jpg","jpeg","png"]:
                    self.images_names_list.append(os.path.join(self.folder_path,name))
            self.next_image()
            
    def next_image(self):
        if self.image_index < len(self.images_names_list)-1:
            self.image_index += 1
            self.img = cv2.imread(self.images_names_list[self.image_index])
            self.img = cv2.cvtColor(self.img,cv2.COLOR_BGR2RGB)
            self.canvas_img = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(self.img))
            self.canvas1.create_image(self.img_width//2,self.img_height//2, image = self.canvas_img, anchor = tk.CENTER)

    def prev_image(self):
        if self.image_index >0:
            self.img = cv2.imread(self.images_names_list[self.image_index])
            self.img = cv2.cvtColor(self.img,cv2.COLOR_BGR2RGB)
            self.image_index -= 1
            self.canvas_img = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(self.img))
            self.canvas1.create_image(self.img_width//2, self.img_height//2, image = self.canvas_img, anchor = tk.CENTER)
    def detect(self):
        self.obj_img = self.show_inference(self.detection_model,self.images_names_list[self.image_index],self.threshold)
        self.canvas_img = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(self.obj_img))
        self.canvas1.create_image(self.img_width//2, self.img_height//2, image = self.canvas_img, anchor = tk.CENTER)
            
    def load_model(self,model_name):
        base_url = 'http://download.tensorflow.org/models/object_detection/'
        model_file = model_name + '.tar.gz'
        model_dir = tf.keras.utils.get_file(
        fname=model_name, 
        origin=base_url + model_file,
        untar=True)
        model_dir = pathlib.Path(model_dir)/"saved_model"
        model = tf.saved_model.load(str(model_dir))
        model = model.signatures['serving_default']
        return model

        
    def run_inference_for_single_image(self,model, image):

        image = np.asarray(image)
        # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
        input_tensor = tf.convert_to_tensor(image)
        # The model expects a batch of images, so add an axis with `tf.newaxis`.
        input_tensor = input_tensor[tf.newaxis,...]
        # Run inference
        output_dict = model(input_tensor)
        # All outputs are batches tensors.
        # Convert to numpy arrays, and take index [0] to remove the batch dimension.
        # We're only interested in the first num_detections.
        num_detections = int(output_dict.pop('num_detections'))
        output_dict = {key:value[0, :num_detections].numpy() 
                     for key,value in output_dict.items()}
        output_dict['num_detections'] = num_detections
        # detection_classes should be ints.
        output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

        return output_dict

    def show_inference(self,model, image_path,threshold):
        # the array based representation of the image will be used later in order to prepare the
        # result image with boxes and labels on it.
        image_np = np.array(Image.open(image_path))
        # Actual detection.
        output_dict = self.run_inference_for_single_image(model, image_np)
        self.label_filter()
        boxes,classes,scores = output_dict['detection_boxes'],output_dict['detection_classes'],output_dict['detection_scores']
        indices = [l for l in range(len(classes)) if classes[l] in self.filter_id]
        
        boxes = np.squeeze(boxes[indices])
        scores = np.squeeze(scores[indices])
        classes = np.squeeze(classes[indices])
        
        # Visualization of the results of a detection.
        if threshold > 1:
            threshold = 1
        if threshold < 0:
            threshold = 0
            
        vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,boxes,classes,scores,
          self.category_index,
          use_normalized_coordinates=True,
          line_thickness=8,min_score_thresh=threshold)
        return image_np


        
app = Runner()
app.mainloop()